# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 


- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [88]:
import mysql.connector 

In [89]:
import config

In [90]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user, 
    passwd = config.passwd,
    database = db_name
)
print(cnx)

In [91]:
## Connect to DB server on AWS
cursor = cnx.cursor()

In [92]:
from mysql.connector import errorcode

In [6]:
## Create new DB 
db_name = "Plumbs"

In [531]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [9]:
# Create a table for the Businesses

db_name = 'Plumbs'

TABLES = {}
TABLES['Plumb_biz'] = (
    "CREATE TABLE Plumb_biz ("
    "  biz_id varchar(40) NOT NULL PRIMARY KEY,"
    "  biz_name varchar(100) NOT NULL,"
    "  review_count int(16) NOT NULL,"
    "  price varchar(11) NOT NULL,"
    "  rating int(7) NOT NULL"
    ") ENGINE=InnoDB")

TABLES['Plumb_Reviews'] = (
    "CREATE TABLE Plumb_Reviews ("
    "  rev_id int(71) NOT NULL PRIMARY KEY,"
    "  rev_rating int(8) NOT NULL,"
    "  name varchar(40) NOT NULL,"
    "  user_id varchar(40) NOT NULL,"
    "  review_text varchar(255) NOT NULL,"
    "  rev_date date NOT NULL,"
    "  biz_id varchar(40) NOT NULL,"
    "  FOREIGN KEY(biz_id) REFERENCES Plumb_biz (biz_id)"
    ") ENGINE=InnoDB")



In [10]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

# cursor.close()
# cnx.close()

Creating table Plumb_biz: already exists.
Creating table Plumb_Reviews: already exists.


In [55]:
cursor.execute("""TRUNCATE Plumb_biz;""")


In [54]:
cursor.execute("""DROP Table Plumb_Reviews;""")

In [ ]:
# Create a table for the reviews

## Part 2: Create ETL pipeline for the business data from the API

In [11]:
# Write a function to make a call to the yelp API
import json
import requests
import pandas as pd
import time

api_key = "NwZvu1WZrjub3P2jL3S_GuZtYSkXkuf2J7zPLOrnPTalaDpzhTVFjNHJW2b2J1r0ILB-XU4qSwnEZ5JckShoShJYrOiOAw6ruSgpsbIxgKv40UuNscGhdAWW7uixXXYx"


term = 'plumber'
location = 'Bushwick NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }


url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)



def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    

    data = response.json()['businesses']
    return data






# def all_results(url_params, api_key):
#     num = response.json()['total']
#     print('{} total matches found.'.format(num))
#     cur = 0
    
#     while cur < num and cur < 1000:
#         url_params['offset'] = cur
#         results = yelp_call(url_params, api_key)
#         parsed_results = parse_results(results)
#         db_insert(plumb_list)
#         time.sleep(1) #Wait a second
#         cur += 50




# df = all_results(url_params, api_key)
# print(len(df))
# df.head()


In [12]:
len(yelp_call(url_params, api_key))

50

In [100]:
num = response.json()['total']
print('{} total matches found.'.format(num))
cur = 0
    
while cur < num and cur < 1000:
    url_params['offset'] = cur
    results = yelp_call(url_params, api_key)
    parsed_results = parse_results(results)
    db_insert(parsed_results)
    time.sleep(1) #Wait a second
    cur += 50

436 total matches found.


TypeError: list indices must be integers or slices, not str

In [99]:
def parse_results(results):
        parsed_results = []
        plumb_list = []
        for result in results:
            yelp_dict = {}
#             for key, value in result.items():
                if key == 'id':
                    yelp_dict['biz_id'] = value
                if key == 'name':
                    yelp_dict['biz_name'] = value
                if key == 'rating':
                    yelp_dict['rating'] = value
                if key == 'review_count':
                    yelp_dict['review_count'] = value
            parsed_results.append(yelp_dict)
        
        for plumb in parsed_results:
            plumb_tuple = (plumb['biz_id'], plumb['biz_name'], 
                     plumb['rating'], plumb['review_count'])
            plumb_list.append(plumb_tuple)
            return plumb_list

IndentationError: unexpected indent (<ipython-input-99-771e993fe8b0>, line 7)

In [104]:
# def all_results(url_params, api_key):
num = response.json()['total']
print('{} total matches found.'.format(num))
cur = 0

while cur < num:
    print(cur)
    url_params['offset'] = cur
    results = yelp_call(url_params, api_key)
    parsed_results = parse_results(results)
    time.sleep(1) #Wait a second
    print(parsed_results[-1])
    db_insert(parsed_results)
    cur += 50
    

436 total matches found.
0


NameError: name 'result' is not defined

In [31]:
all_results(url_params, api_key)

436 total matches found.


IntegrityError: 1062 (23000): Duplicate entry 'tXtXY7r8NXPk0-MKJlrOjQ' for key 'PRIMARY'

In [522]:
locale = 'enUS'
url_params = { 'local': locale}

url = 'https://api.yelp.com/v3/businesses/{o0WJDT2-EXDUFNtmQ2wEqA}/reviews'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)

In [529]:
import json
import requests
import pandas as pd

api_key = "NwZvu1WZrjub3P2jL3S_GuZtYSkXkuf2J7zPLOrnPTalaDpzhTVFjNHJW2b2J1r0ILB-XU4qSwnEZ5JckShoShJYrOiOAw6ruSgpsbIxgKv40UuNscGhdAWW7uixXXYx"

def yelp_call_r(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/o0WJDT2-EXDUFNtmQ2wEqA/reviews'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    

    data_r = response.json()['reviews']
    return data_r

url = 'https://api.yelp.com/v3/businesses/o0WJDT2-EXDUFNtmQ2wEqA/reviews'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)


def all_results_r(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
#         db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

import time

# term = 'plumber'
# location = 'Bushwick NY'
# url_params = {  'term': term.replace(' ', '+'),
#                 'location': location.replace(' ', '+'),
#                 'limit' : 50
#              }
df_2r = all_results(url_params, api_key)
print(len(df_2r))
# df.head()

1 total matches found.


TypeError: object of type 'NoneType' has no len()

In [94]:
dbi_1 = yelp_call(url_params, api_key)

In [95]:
dbi_1

[{'id': 'tXtXY7r8NXPk0-MKJlrOjQ',
  'alias': 'friedman-irwin-and-son-brooklyn',
  'name': 'Friedman Irwin & Son',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/jD4myr5l9ob1aSe7rCFq1w/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/friedman-irwin-and-son-brooklyn?adjust_creative=RD6sXbiK7pfyZyJHpmys3w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=RD6sXbiK7pfyZyJHpmys3w',
  'review_count': 50,
  'categories': [{'alias': 'plumbing', 'title': 'Plumbing'}],
  'rating': 4.5,
  'coordinates': {'latitude': 40.72369, 'longitude': -73.95462},
  'transactions': [],
  'location': {'address1': '139 Banker St',
   'address2': '',
   'address3': '',
   'city': 'Brooklyn',
   'zip_code': '11222',
   'country': 'US',
   'state': 'NY',
   'display_address': ['139 Banker St', 'Brooklyn, NY 11222']},
  'phone': '+17187827150',
  'display_phone': '(718) 782-7150',
  'distance': 4203.5129390794245},
 {'id': '1M5qDSvsfNogoZeXYx2icg',
  'alias': 'pipe-monkeys

In [116]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB
# def parse_results(results):
#         parsed_results = []
#         for result in results:
#             yelp_dict = {}
#             for key, value in result.items():
#                 if key == 'id':
#                     yelp_dict['biz_id'] = value
#                 if key == 'name':
#                     yelp_dict['biz_name'] = value
#                 if key == 'rating':
#                     yelp_dict['rating'] = value
#                 if key == 'review_count':
#                     yelp_dict['review_count'] = value
#             parsed_results.append(yelp_dict)
                
#         return parsed_results
    
def parse_results(results):
    fields = ['id', 'name', 'rating', 'review_count']
    parsed_results = []
    for result in results:
        parsed_result = {field: (result[field] in result.keys())
                            for field in fields}
        parsed_results.append(parsed_results)
        return parsed_results

In [117]:
parse_results(yelp_call(url_params, api_key))

[[...]]

In [119]:
dbi_v2 = parse_results(dbi_1) 
dbi_v2

[[...]]

In [118]:
# Write a function to take your parsed data and insert it into the DB
plumb_list = []
for plumb in dbi_v2:
  
    plumb_tuple = (plumb['biz_id'], plumb['biz_name'], 
                     plumb['review_count'], plumb['rating'])
    plumb_list.append(plumb_tuple)
    
plumb_list



TypeError: tuple indices must be integers or slices, not str

In [21]:
len(plumb_list)

50

In [22]:
len(set(plumb_list))

50

In [23]:
plumb_list[49]

('nshCt83YkUseutjNzidTNg', '39.99 sewer service', 7, 2.0)

In [39]:

insert_statement = "INSERT INTO Plumb_biz (biz_id, biz_name, review_count, rating)  VALUES (%s, %s, %s, %s)"



In [19]:
cursor.executemany (insert_statement, plumb_list)

cnx.commit()

In [97]:
def db_insert(parsed_results):
#     plumb_list = []
    plumb_tuples = [(parsed_results['biz_id'],
                    parsed_results['biz_name'],
                    parsed_results['review_count'],
                    parsed_results['rating'])
                   for parsed_result in parsed_results]
    insert_statement = "INSERT INTO Plumb_biz (biz_id, biz_name, review_count, rating)  VALUES (%s, %s, %s, %s)"
    cursor.executemany(insert_statement, plumb_tuples)
    cnx.commit()
    
    
#     plumb_list = []
# for plumb in dbi_v2:
  
#     plumb_tuple = (plumb['biz_id'], plumb['biz_name'], 
#                      plumb['review_count'], plumb['rating'])
#     plumb_list.append(plumb_tuple)

In [53]:
plumb_tuples = [(dbi_1['biz_id'],
                dbi_1['biz_name'],
                dbi_1['review_count'],
                dbi_1['rating'])
               for i in dbi_1]

plumb_tuples

TypeError: list indices must be integers or slices, not str

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [479]:
cnx

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews


In [ ]:
# Write a function to parse out the relevant information from the reviews

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 4: Write SQL queries that will answer the questions posed. 

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
def parse_results(results):
    
    return parsed_results

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time



def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)